In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
class LSTMRecommendationModel(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim, hidden_dim):
        super(LSTMRecommendationModel, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        
        # Item Embedding layer
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        
        # LSTM Layer
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,batch_first=True)
        
        # Outlet Layer
        self.output_layer = nn.Linear(hidden_dim, num_items)
        
    def forward(self, sequences):
        # Input: sequences of user-item interactins
        embedded_sequences = self.item_embedding(sequences)
        
        # LSTM forward pass
        lstm_out, _ = self.lstm(embedded_sequences)
        
        # output layer
        output = self.output_layer(lstm_out[:, -1, :]) # Taking the last timestep's output
        return output

In [3]:
num_users = 5
num_items = 6
embedding_dim = 32
hidden_dim = 64

In [4]:
interaction_matrix = torch.tensor([[1, 0, 1, 0, 0, 1],
                                   [0, 1, 0, 1, 0, 0],
                                   [1, 0, 0, 0, 1, 0],
                                   [0, 1, 1, 0, 1, 0],
                                   [0, 0, 0, 1, 0, 1]])

In [5]:
sequences = interaction_matrix.unsqueeze(0) # Add batch dimension
sequences = sequences.unsqueeze(-1)
sequences = interaction_matrix

In [6]:
model = LSTMRecommendationModel(num_users, num_items, embedding_dim, hidden_dim)


In [7]:
output = model(sequences)

In [8]:
print(output)

tensor([[-0.1601,  0.0408,  0.1101, -0.1443,  0.0295, -0.0782],
        [-0.0757, -0.0250,  0.1054, -0.1724,  0.0217, -0.1243],
        [-0.0933,  0.0025,  0.1084, -0.1748,  0.0088, -0.1182],
        [-0.1099,  0.0263,  0.1068, -0.1647, -0.0022, -0.1180],
        [-0.1664,  0.0552,  0.1132, -0.1447,  0.0256, -0.0736]],
       grad_fn=<AddmmBackward0>)


In [9]:
# Define loss functoin and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [11]:
# Trainig loop
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(sequences)
    
    # Compute loss
    loss = criterion(outputs.squeeze(), torch.tensor([0,1,2,3,4]))
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Print progress
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 1.6554
Epoch [20/100], Loss: 1.5348
Epoch [30/100], Loss: 1.4116
Epoch [40/100], Loss: 1.2831
Epoch [50/100], Loss: 1.1620
Epoch [60/100], Loss: 1.0555
Epoch [70/100], Loss: 0.9580
Epoch [80/100], Loss: 0.8605
Epoch [90/100], Loss: 0.7601
Epoch [100/100], Loss: 0.6574


In [21]:
def generate_recommendation(model, user_id, num_items_to_recommend=3, popularity_items=None):
    interaction_matrix = torch.tensor([[1, 0, 1, 0, 0, 1],  # Sample interaction matrix
                                       [0, 1, 0, 1, 0, 0],
                                       [1, 0, 0, 0, 1, 0],
                                       [0, 1, 1, 0, 1, 0],
                                       [0, 0, 0, 1, 0, 1]])
    
    if user_id >= len(interaction_matrix):
        # Handle cold start for new users by providing popularity-based recommendations
        if popularity_items is None:
            raise ValueError("Popularity items list must be provided for cold start handling.")
        return popularity_items[:num_items_to_recommend]
    
    user_sequence = interaction_matrix[user_id].unsqueeze(0)
    
    # forward pass through the model to get predictions
    with torch.no_grad():
        predictions = model(user_sequence)
        
    # Convert predictions to probabilities using softmax
    probabilities = torch.softmax(predictions, dim=1)
    
    # Get top recommended items
    _, top_indices = torch.topk(probabilities, num_items_to_recommend, dim=1)
    
    # convert indeices to item IDs
    top_item_ids = top_indices.squeeze().tolist()
    return top_item_ids

In [23]:
# Example usage:
# Load the trained model (already trained in the previous example)
# model = LSTMRecommendationModel(num_users, num_items, embedding_dim, hidden_dim)
# model.load_state_dict(torch.load('lstm_recommendation_model.pth'))  # Load saved model state

# Sample popularity-based items list (for cold start handling)
popularity_items = [0, 1, 2, 3]

# Generate recommendations for user 0
user_id = [0,1,2,3,10]
for user in user_id:
    recommended_items = generate_recommendation(model, user, popularity_items=popularity_items)
    print(f"Recommended items for user {user}: {recommended_items}")

Recommended items for user 0: [0, 4, 3]
Recommended items for user 1: [1, 2, 3]
Recommended items for user 2: [2, 1, 3]
Recommended items for user 3: [3, 2, 1]
Recommended items for user 10: [0, 1, 2]


In [24]:
def train_model(model, train_data, criterion, optimizer, num_epochs=10):
    # Set the model to training mode
    model.train()
    
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        
        # Iterate over the training data
        for sequences, targets in train_data:
            # Forward pass
            outputs = model(sequences)
            
            # Compute the loss
            loss = criterion(outputs, targets)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Accumulate the loss for the epoch
            epoch_loss += loss.item()
        
        # Calculate average epoch loss
        avg_epoch_loss = epoch_loss / len(train_data)
        
        # Print epoch loss
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_epoch_loss:.4f}')

In [25]:
from torch.utils.data import Dataset, DataLoader

# Define a custom dataset class to represent the training data
class RecommendationDataset(Dataset):
    def __init__(self, interaction_matrix):
        self.interaction_matrix = interaction_matrix
    
    def __len__(self):
        return len(self.interaction_matrix)
    
    def __getitem__(self, idx):
        user_sequence = self.interaction_matrix[idx]
        target_item = user_sequence[-1]  # Target is the last item in the sequence
        input_sequence = user_sequence[:-1]  # Input sequence excludes the target item
        return input_sequence, target_item

# Example interaction matrix (user-item interaction data)
interaction_matrix = torch.tensor([[1, 0, 1, 0, 0, 1],
                                   [0, 1, 0, 1, 0, 0],
                                   [1, 0, 0, 0, 1, 0],
                                   [0, 1, 1, 0, 1, 0],
                                   [0, 0, 0, 1, 0, 1]])

# Create an instance of the custom dataset
train_dataset = RecommendationDataset(interaction_matrix)

# Define the DataLoader to iterate over batches of data
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

# Sample usage of train_loader
for input_sequence, target_item in train_loader:
    print("Input sequence:", input_sequence)
    print("Target item:", target_item)
    break  # Print only the first batch

Input sequence: tensor([[1, 0, 1, 0, 0]])
Target item: tensor([1])


In [26]:
train_model(model, train_loader, criterion, optimizer, num_epochs=10)

Epoch [1/10], Loss: 1.2362
Epoch [2/10], Loss: 0.6799
Epoch [3/10], Loss: 0.4424
Epoch [4/10], Loss: 0.5855
Epoch [5/10], Loss: 0.3185
Epoch [6/10], Loss: 0.3068
Epoch [7/10], Loss: 0.2591
Epoch [8/10], Loss: 0.2916
Epoch [9/10], Loss: 0.1757
Epoch [10/10], Loss: 0.1258
